In [1]:
#Python version
!python --version

Python 3.8.5


In [2]:
#Install uninstalled modules
#!pip install ipykernel
!pip install MetaTrader5==5.0.34
!pip install gym
!pip install gym-anytrading
!pip install tensorflow

In [3]:
#Import needed modules
import gym
import gym_anytrading
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
import datetime
import MetaTrader5 as mt5

In [4]:
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    mt5.shutdown()
 
# display data on MetaTrader 5 version
print(mt5.version())

weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

"""
# Initialize Metatrader on a certain day
if datetime.datetime.now() == weekdays:
    # now connect to another trading account specifying the password
    account = #Account number
    password = #Password number
    server = #Server name
    authorized=mt5.login(account, password, server)
    if authorized:
        # display trading account data 'as is'
        print(mt5.account_info())
        # display trading account data in the form of a list
        print("Show account_info()._asdict():")
        account_info_dict = mt5.account_info()._asdict()
        for prop in account_info_dict:
            print("  {}={}".format(prop, account_info_dict[prop]))
    else:
        print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
# shut down connection to the MetaTrader 5 terminal
else:
    mt5.shutdown()
"""

MetaTrader5 package author:  MetaQuotes Software Corp.
MetaTrader5 package version:  5.0.34
initialize() failed, error code = (-10003, 'IPC initialize failed, MetaTrader 5 x64 not found')
(0, 0, '')


'\n# Initialize Metatrader on a certain day\nif datetime.datetime.now() == weekdays:\n    # now connect to another trading account specifying the password\n    account = #Account number\n    password = #Password number\n    server = #Server name\n    authorized=mt5.login(account, password, server)\n    if authorized:\n        # display trading account data \'as is\'\n        print(mt5.account_info())\n        # display trading account data in the form of a list\n        print("Show account_info()._asdict():")\n        account_info_dict = mt5.account_info()._asdict()\n        for prop in account_info_dict:\n            print("  {}={}".format(prop, account_info_dict[prop]))\n    else:\n        print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))\n# shut down connection to the MetaTrader 5 terminal\nelse:\n    mt5.shutdown()\n'

In [5]:
custom_env = gym.make('forex-v0',
               df = FOREX_EURUSD_1H_ASK,
               window_size = 10,
               frame_bound = (10, 300),
               unit_side = 'right')

print()
print("custom_env information:")
print("> shape:", custom_env.shape)
print("> df.shape:", custom_env.df.shape)
print("> prices.shape:", custom_env.prices.shape)
print("> signal_features.shape:", custom_env.signal_features.shape)
print("> max_possible_profit:", custom_env.max_possible_profit())

#custom_env.reset() ##Uncomment if you need it.
#custom_env.render()


custom_env information:
> shape: (10, 2)
> df.shape: (6225, 5)
> prices.shape: (300,)
> signal_features.shape: (300, 2)
> max_possible_profit: 1.122900180008982


In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32, input_dim = 4, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation = "softmax"))
model.build()
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [7]:
def discount_rewards(r, gamma = 0.8):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [8]:
env = gym.make('CartPole-v0')
episodes = 2000
scores = []
update_every = 5

gradBuffer = model.trainable_variables
for ix,grad in enumerate(gradBuffer):
  gradBuffer[ix] = grad * 0
  
for e in range(episodes):
  
  s = env.reset()
  
  env.render()
  ep_memory = []
  ep_score = 0
  done = False 
  while not done: 
    s = s.reshape([1,4])
    with tf.GradientTape() as tape:
      #forward pass
      logits = model(s)
      a_dist = logits.numpy()
      # Choose random action with p = action dist
      a = np.random.choice(a_dist[0],p=a_dist[0])
      a = np.argmax(a_dist == a)
      loss = compute_loss([a], logits)
    # make the choosen action 
    s, r, done, _ = env.step(a)
    ep_score +=r
    if done: r-=10
    grads = tape.gradient(loss, model.trainable_variables)
    ep_memory.append([grads,r])
  scores.append(ep_score)
  # Discound the rewards 
  ep_memory = np.array(ep_memory)
  ep_memory[:,1] = discount_rewards(ep_memory[:,1])
  
  for grads, r in ep_memory:
    for ix,grad in enumerate(grads):
      gradBuffer[ix] += grad * r
  
  if e % update_every == 0:
    optimizer.apply_gradients(zip(gradBuffer, model.trainable_variables))
    for ix,grad in enumerate(gradBuffer):
      gradBuffer[ix] = grad * 0
      
  if e % 100 == 0:
    print("Episode  {}  Score  {}".format(e, np.mean(scores[-100:])))

env.close()

<ipython-input-8-9a06370b7277>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ep_memory = np.array(ep_memory)
Episode  0  Score  12.0
Episode  100  Score  35.76
Episode  200  Score  106.08
Episode  300  Score  126.79
Episode  400  Score  177.98
Episode  500  Score  166.72
Episode  600  Score  73.29
Episode  700  Score  180.04
Episode  800  Score  200.0
Episode  900  Score  199.74
Episode  1000  Score  200.0
Episode  1100  Score  197.49
Episode  1200  Score  198.48
Episode  1300  Score  195.06
Episode  1400  Score  187.24
Episode  1500  Score  159.27
Episode  1600  Score  125.85
Episode  1700  Score  181.43
Episode  1800  Score  200.0
Episode  1900  Score  199.1
